In [1]:
import pandas as pd
from collections import Counter
import numpy as np
from copy import deepcopy
import random
from functools import reduce
import pickle
from sklearn.model_selection import train_test_split

In [6]:
train_replace_dict = {'atis_flight#atis_airfare': 'atis_airfare', 
                                     'atis_ground_service#atis_ground_fare': 'atis_ground_fare',
                                     'atis_aircraft#atis_flight#atis_flight_no': 'atis_flight_no',
                                     'atis_airfare#atis_flight_time':'atis_flight_time',
                                     'atis_airline#atis_flight_no':'atis_flight_no',
                                     'atis_flight_no#atis_airline' : 'atis_flight_no',
                                     'atis_airfare#atis_flight': 'atis_airfare',
                                     'atis_flight#atis_airline' : 'atis_airline'}

# intent to idx mapping
intent_list = (pd.read_csv('../../data/multiATIS/intent_list.csv',sep=',',header=None,names=['INTENT']).INTENT.values.tolist())
intent2idx = { intent:idx for idx,intent in enumerate(intent_list)}

# slot idx mapping
final_slots = pd.read_csv('../../data/multiATIS/slots_list.csv',sep=',',header=None,names=['SLOTS']).SLOTS.values.tolist()
slots2idx  = {slots:idx for idx,slots in enumerate(final_slots)}
idx2slots = {v: k for k, v in slots2idx.items()}

In [7]:
def mapping2idx(df):

    encoded_intent = []
    for intent in list(df.INTENT):
        try:
            encoded_intent.append(intent2idx[intent])
        except:
            print(intent)
            encoded_intent.append(intent2idx['OOI'])
            
        
    df['INTENT_ID'] = encoded_intent 

    # adding mapped slot column
    slots_ID = []

    for annotations in list(df.SLOTS):
        encoded_slot = ""
        #print((annotations.split()))
        for tokens in annotations.split():
            try: 
                encoded_slot += str(slots2idx[tokens]) + " "
            except:
                print('token',tokens)
                encoded_slot += str(slots2idx['O']) + " "
        slots_ID.append(" ".join(encoded_slot.split()))

    df['SLOTS_ID'] = slots_ID
    
    return df

In [8]:
def get_carrier_phrase_length(annotations):
    CP_idx = []
    for idx,token in enumerate(annotations.split(' ')):
        if token == 'O':
            CP_idx.append(idx)
    
    return CP_idx 

In [9]:
def carrier_aug(data,tau):
    
    orig_data = deepcopy(data)
    
    augINTENT, augSLOTS, augTEXT,augID = [],[],[],[]
    
    cnt = 0
    for sample in data.values.tolist():
        
        
        CP_idx = get_carrier_phrase_length(sample[2])
        
        CP_length = len(CP_idx)
        
        if CP_length <= 2:
            
            augINTENT.append(sample[3])
            augTEXT.append(sample[1])
            augSLOTS.append(sample[2])
            augID.append(cnt)
            cnt+=1
        else:
            del_count = int(CP_length/2) if CP_length <= 5 else int(tau*CP_length)
        
            del_index = random.sample(CP_idx,del_count)

            TEXT = ' '.join([i for j, i in enumerate(sample[1].split(' ')) if j not in del_index])
            SLOTS = ' '.join([i for j, i in enumerate(sample[2].split(' ')) if j not in del_index])
         
            augINTENT.append(sample[3])
            augTEXT.append(TEXT)
            augSLOTS.append(SLOTS)
            augID.append(cnt)
            cnt+=1
    
               
    augPD = pd.DataFrame([augID,augTEXT,augSLOTS,augINTENT],index=['ID','TEXT','SLOTS','INTENT']).T
    
    return augPD

In [14]:
def OOC_aug(data_path,out_path,fn,tau):
    
    # loading dataset
    data = pd.read_csv(data_path,sep='\t',header=0,names=['ID','TEXT','SLOTS','INTENT'])
    
    # lowerCasing the TEXT column
    data['TEXT'] = data['TEXT'].str.lower()
    
    # handling multi-label instances 
    data['INTENT'] = data['INTENT'].replace(train_replace_dict)
    
    aug_data = carrier_aug(data,tau=tau)
    
    aug_data = mapping2idx(aug_data)
    
    aug_data = aug_data.sample(frac=1).reset_index(drop=True)
    aug_data.to_csv(out_path+fn,sep='\t',index=False)
    

In [16]:
OOC_aug('../../data/multiATIS/raw/train_dev_test/test_EN.tsv','../../data/multiATIS/split/test/OOC_Noise/20per/','test_EN_01.tsv',tau=0.10)
OOC_aug('../../data/multiATIS/raw/train_dev_test/test_EN.tsv','../../data/multiATIS/split/test/OOC_Noise/20per/','test_EN_02.tsv',tau=0.10)
OOC_aug('../../data/multiATIS/raw/train_dev_test/test_EN.tsv','../../data/multiATIS/split/test/OOC_Noise/20per/','test_EN_03.tsv',tau=0.10)
OOC_aug('../../data/multiATIS/raw/train_dev_test/test_EN.tsv','../../data/multiATIS/split/test/OOC_Noise/20per/','test_EN_04.tsv',tau=0.10)
OOC_aug('../../data/multiATIS/raw/train_dev_test/test_EN.tsv','../../data/multiATIS/split/test/OOC_Noise/20per/','test_EN_05.tsv',tau=0.10)
OOC_aug('../../data/multiATIS/raw/train_dev_test/test_EN.tsv','../../data/multiATIS/split/test/OOC_Noise/40per/','test_EN_01.tsv',tau=0.25)
OOC_aug('../../data/multiATIS/raw/train_dev_test/test_EN.tsv','../../data/multiATIS/split/test/OOC_Noise/40per/','test_EN_02.tsv',tau=0.25)
OOC_aug('../../data/multiATIS/raw/train_dev_test/test_EN.tsv','../../data/multiATIS/split/test/OOC_Noise/40per/','test_EN_03.tsv',tau=0.25)
OOC_aug('../../data/multiATIS/raw/train_dev_test/test_EN.tsv','../../data/multiATIS/split/test/OOC_Noise/40per/','test_EN_04.tsv',tau=0.25)
OOC_aug('../../data/multiATIS/raw/train_dev_test/test_EN.tsv','../../data/multiATIS/split/test/OOC_Noise/40per/','test_EN_05.tsv',tau=0.25)
OOC_aug('../../data/multiATIS/raw/train_dev_test/test_EN.tsv','../../data/multiATIS/split/test/OOC_Noise/60per/','test_EN_01.tsv',tau=0.50)
OOC_aug('../../data/multiATIS/raw/train_dev_test/test_EN.tsv','../../data/multiATIS/split/test/OOC_Noise/60per/','test_EN_02.tsv',tau=0.50)
OOC_aug('../../data/multiATIS/raw/train_dev_test/test_EN.tsv','../../data/multiATIS/split/test/OOC_Noise/60per/','test_EN_03.tsv',tau=0.50)
OOC_aug('../../data/multiATIS/raw/train_dev_test/test_EN.tsv','../../data/multiATIS/split/test/OOC_Noise/60per/','test_EN_04.tsv',tau=0.50)
OOC_aug('../../data/multiATIS/raw/train_dev_test/test_EN.tsv','../../data/multiATIS/split/test/OOC_Noise/60per/','test_EN_05.tsv',tau=0.50)


atis_day_name
atis_day_name
token B-compartment
token B-stoploc.airport_code
token B-booking_class
token B-flight
atis_day_name
atis_day_name
token B-compartment
token B-stoploc.airport_code
token B-booking_class
token B-flight
atis_day_name
atis_day_name
token B-compartment
token B-stoploc.airport_code
token B-booking_class
token B-flight
atis_day_name
atis_day_name
token B-compartment
token B-stoploc.airport_code
token B-booking_class
token B-flight
atis_day_name
atis_day_name
token B-compartment
token B-stoploc.airport_code
token B-booking_class
token B-flight
atis_day_name
atis_day_name
token B-compartment
token B-stoploc.airport_code
token B-booking_class
token B-flight
atis_day_name
atis_day_name
token B-compartment
token B-stoploc.airport_code
token B-booking_class
token B-flight
atis_day_name
atis_day_name
token B-compartment
token B-stoploc.airport_code
token B-booking_class
token B-flight
atis_day_name
atis_day_name
token B-compartment
token B-stoploc.airport_code
token B-boo